In [1]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

# Check env

In [3]:
env = gym.make('CartPole-v1')

In [4]:
observation = env.reset()
observation[0]

array([0.03549822, 0.03111971, 0.03809689, 0.04066058], dtype=float32)

# Replay Buffer

In [6]:
class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, experience):
        '''
        experience should be [1,5], (st, at, rt, st+1, done)
        done should be 1/0
        '''
        self.buffer.append(experience)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

# QNetwork


In [18]:
torch.tensor(observation[0]).shape

torch.Size([4])

In [ ]:
# Check Forward
Q = Qnet()
out = Q(torch.tensor(observation[0]))
print(out)
a = Q.sample_action(torch.tensor(observation[0]), 0.1)
print(a)

In [7]:
class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        # 0. Init an NN

        ##################################
        #         Your Answer            #
        ##################################

    def forward(self, x):
        # 1. forward input data

        ##################################
        #         Your Answer            #
        ##################################
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        # Get an reward func with greedy epsilon, choose an action

        ##################################
        #         Your Answer            #
        ##################################
        pass
        return at    # should be a scaler
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        
        # 0. Sample from memory
        ##################################
        #         Your Answer            #
        ##################################

        # 1.1 calculate Q
        ##################################
        #         Your Answer            #
        ##################################

        # 1.2 calculate y, based on theta-
        ##################################
        #         Your Answer            #
        ##################################
        pass
        # 1.3 loss_func
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Learning

In [ ]:
# STEP 0: Initialize, (1)D, (2)Q
memory = ReplayBuffer()
Q = Qnet()

In [10]:
s_prime, r, done, _, _ = env.step(1)

print(s_prime, r, done)

done_mask = 0.0 if done else 1.0

[ 0.01404168  0.0211521  -0.04275841 -0.03881681] 1.0 False


In [ ]:
def dqn(env, q, memory, max_epi=1000, min_memory=2000):

    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())

    print_interval = 20
    score = 0.0  
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)

    # iter for episodes
    for n_epi in range(max_epi):
        epsilon = max(0.01, 0.08 - 0.01*(n_epi/200)) #Linear annealing from 8% to 1%
        s, _ = env.reset()
        done = False
        
        # iter for steps, util terminate
        while not done:

            # STEP1: Sample (st,at,rt,st+1), and store (st,at,rt,st+1,done)
            ##################################
            #         Your Answer            #
            ##################################
            print()

            score += r
            if done:
                break
        
        # STEP 2: update until larger than 2000 samples
        if memory.size()>min_memory:
            continue
            train(q, q_target, memory, optimizer)

        if n_epi%print_interval==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
            print("n_episode :{}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(
                                                            n_epi, score/print_interval, memory.size(), epsilon*100))
            score = 0.0
            
    return q

In [ ]:
dqn(env, Q, memory)